In [ ]:
# Import necessary libraries
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install numpy

import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

# Read the sales data from a CSV file
sales_df = pd.read_csv('/Users/user/Desktop/data_science/sales.csv', index_col=[0])
sales_df.head()

# Read the stock levels data from a CSV file
stock_df = pd.read_csv('/Users/user/Desktop/data_science/sensor_stock_levels.csv', index_col=[0])
stock_df.head()

# Read the temperature data from a CSV file
temp_df = pd.read_csv('/Users/user/Desktop/data_science/sensor_storage_temperature.csv', index_col=[0])
temp_df.head()

# Display information about the sales data
sales_df.info()

# Display information about the stock levels data
stock_df.info()

# Display information about the temperature data
temp_df.info()

# Function to convert a column to datetime format
def convert_to_datetime(data, column):
    dummy = data.copy()
    dummy[column] = pd.to_datetime(dummy[column], format='%Y-%m-%d %H:%M:%S')
    return dummy

# Convert the timestamp column in sales data to datetime format
sales_df = convert_to_datetime(sales_df, 'timestamp')
sales_df.info()

# Convert the timestamp column in stock levels data to datetime format
stock_df = convert_to_datetime(stock_df, 'timestamp')
stock_df.info()

# Convert the timestamp column in temperature data to datetime format
temp_df = convert_to_datetime(temp_df, 'timestamp')
temp_df.info()

# Function to convert timestamp to hourly format
def convert_timestamp_to_hourly(data, column):
    dummy = data.copy()
    new_ts = dummy[column].tolist()
    new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
    new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
    dummy[column] = new_ts
    return dummy

# Convert timestamp to hourly format in sales data
sales_df = convert_timestamp_to_hourly(sales_df, 'timestamp')
sales_df.head()

# Convert timestamp to hourly format in stock levels data
stock_df = convert_timestamp_to_hourly(stock_df, 'timestamp')
stock_df.head()

# Convert timestamp to hourly format in temperature data
temp_df = convert_timestamp_to_hourly(temp_df, 'timestamp')
temp_df.head()

# Aggregate sales data by timestamp and product_id, summing up quantities
sales_agg = sales_df.groupby(['timestamp', 'product_id']).agg({'quantity':'sum'}).reset_index()
sales_agg.head()

# Aggregate stock levels data by timestamp and product_id, calculating the mean of estimated stock percentages
stock_agg = stock_df.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
stock_agg.head()

# Aggregate temperature data by timestamp, calculating the mean temperature
temp_agg = temp_df.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()
temp_agg.head()

# Merge the stock levels and sales data on timestamp and product_id
merged_df = stock_agg.merge(sales_agg, on=['timestamp', 'product_id'], how='left')
merged_df.head()

# Merge the merged data with temperature data on timestamp
merged_df = merged_df.merge(temp_agg, on='timestamp', how='left')
merged_df.head()

# Display information about the merged data
merged_df.info()

# Fill missing quantity values with 0
merged_df['quantity'] = merged_df['quantity'].fillna(0)
merged_df.info()

# Extract product categories and product prices from sales data
product_categories = sales_df[['product_id', 'category']]
product_categories = product_categories.drop_duplicates()

product_price = sales_df[['product_id', 'unit_price']]
product_price = product_price.drop_duplicates()

# Merge product categories and prices into the merged data
merged_df = merged_df.merge(product_categories, on="product_id", how="left")
merged_df.head()

merged_df = merged_df.merge(product_price, on="product_id", how="left")
merged_df.head()

# Display information about the merged data after adding product information
merged_df.info()

# Extract day of the month, day of the week, and hour from the timestamp
merged_df['timestamp_day_of_month'] = merged_df['timestamp'].dt.day
merged_df['timestamp_day_of_week'] = merged_df['timestamp'].dt.dayofweek
merged_df['timestamp_hour'] = merged_df['timestamp'].dt.hour

# Drop the timestamp column
merged_df.drop(columns=['timestamp'], inplace=True)
merged_df.head()

# Perform one-hot encoding for the product categories
merged_df = pd.get_dummies(merged_df, columns=['category'])
merged_df.head()

# Drop the product_id column
merged_df.drop(columns=['product_id'], inplace=True)
merged_df.head()

# Separate features (X) and target (y)
X = merged_df.drop(columns=['estimated_stock_pct'])
y = merged_df['estimated_stock_pct']
print(X.shape)
print(y.shape)

# Define the number of folds and train-test split ratio
K = 10
split = 0.75

accuracy = []

# Perform K-fold cross-validation
for fold in range(0, K):

    # Instantiate the RandomForestRegressor model and the StandardScaler
    model = RandomForestRegressor()
    scaler = StandardScaler()

    # Create training and test samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=split, random_state=42)

    # Scale the features using StandardScaler
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    trained_model = model.fit(X_train, y_train)

    # Generate predictions on the test sample
    y_pred = trained_model.predict(X_test)

    # Compute the mean absolute error (MAE)
    mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
    accuracy.append(mae)
    print(f"Fold {fold + 1}: MAE = {mae:.3f}")

# Calculate and display the average MAE
print(f"Average MAE: {(sum(accuracy) / len(accuracy)):.2f}")

# Visualize feature importances
features = [i.split("__")[0] for i in X.columns]
importances = model.feature_importances_
indices = np.argsort(importances)

fig, ax = plt.subplots(figsize=(10, 20))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()